In [4]:
import pandas as pd
import geopandas as gpd

In [7]:
id_id_relation = pd.read_csv("../model/id_id_relation.csv", dtype=str)
id_id_relation.head()

,SOURCE_ID,TARGET_TYPE,SOURCE_LAT,SOURCE_LONG,TARGET_ID,TARGET_TYPE.1,TARGET_LAT,TARGET_LONG,TYPE,RELATION
0,1,A,35.6814279,139.7671784,1,B,35.6814279,139.7671784,A->B,100
1,1,A,35.6814279,139.7671784,2,A,35.7140332,139.7770907,A->A,40
2,2,A,35.7140332,139.7770907,1,A,35.6814279,139.7671784,A->A,20
3,3,A,35.68964035,139.7005604,1,B,35.6814279,139.7671784,A->B,50


In [33]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
import folium

# CSVデータの読み込み
data = pd.read_csv("../model/id_id_relation.csv")

# Shapelyを使用してLineStringを作成
data['geometry'] = data.apply(lambda row: LineString([(row['SOURCE_LONG'], row['SOURCE_LAT']), (row['TARGET_LONG'], row['TARGET_LAT'])]), axis=1)

# GeoDataFrameに変換
gdf = gpd.GeoDataFrame(data, geometry='geometry', crs='4326')

# タイプのユニークな値を取得
unique_types = gdf['TYPE'].unique()

# 固定のカラーパレットを定義
color_palette = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
]

# カラーパレットの色をユニークなタイプに割り当て
color_dict = {unique_types[i]: color_palette[i % len(color_palette)] for i in range(len(unique_types))}
gdf['color'] = gdf['TYPE'].map(color_dict)

# relation列の標準化
min_relation = gdf['RELATION'].min()
max_relation = gdf['RELATION'].max()
gdf['weight'] = (gdf['RELATION'] - min_relation) / (max_relation - min_relation)*10.0 + 1.0

# gdf["style"] = {"weight": 2, "color": gdf['TYPE'].map(color_dict)}

# Foliumマップの作成
m = folium.Map(location=[35.6814279, 139.7671784], zoom_start=12)

# 各タイプのレイヤーを作成
for type_ in unique_types:
    layer = folium.FeatureGroup(name=type_)
    type_gdf = gdf[gdf['TYPE'] == type_]

    # GeoJSONをマップに追加
    folium.GeoJson(
        type_gdf,
        style_function=lambda feature: {
            'color': feature['properties']['color'],
            'weight': feature['properties']['weight']
        }
    ).add_to(layer)
    layer.add_to(m)


layer = folium.FeatureGroup(name="loc")
loc_df = gdf[["SOURCE_ID", "TARGET_TYPE", "SOURCE_LAT", "SOURCE_LONG"]].copy()
loc_df['geometry'] = loc_df.apply(lambda row: Point([row['SOURCE_LONG'], row['SOURCE_LAT']]), axis=1)

# GeoDataFrameに変換
loc_gdf = gpd.GeoDataFrame(loc_df, geometry='geometry', crs='4326')

# GeoJSONをマップに追加
folium.GeoJson(
    loc_gdf,
    # style_function=lambda feature: {
    #     'color': feature['properties']['color'],
    #     'weight': feature['properties']['weight']
    # }
).add_to(layer)
layer.add_to(m)

# LayerControlを追加
folium.LayerControl().add_to(m)





# # タイプによる色分けの辞書
# color_dict = {
#     'A->A': 'blue',
#     'A->B': 'green'
# }

# # GeoDataFrameの属性を元にGeoJSONのスタイルを設定
# gdf['color'] = gdf['TYPE'].map(color_dict)
# gdf['weight'] = gdf['RELATION'] / 10

# # GeoDataFrameをGeoJSONに変換
# geojson = gdf.to_json()

# # Foliumマップの作成
# m = folium.Map(location=[35.6814279, 139.7671784], zoom_start=12)

# # GeoJSONをマップに追加
# folium.GeoJson(
#     geojson,
#     style_function=lambda feature: {
#         'color': feature['properties']['color'],
#         'weight': feature['properties']['weight']
#     }
# ).add_to(m)

# マップを保存
m.save("map.html")

